# Agent Prompt and Context Engineering

Master advanced prompt engineering techniques and context management for optimal agent performance.

## Why Prompt Engineering Matters

The system prompt is the **most important** configuration for agent behavior:
- Defines agent role and personality
- Guides decision-making and tool selection
- Sets response format and style
- Establishes boundaries and constraints

**Key Principles:**
1. **Be specific** - Vague prompts lead to unpredictable behavior
2. **Provide examples** - Show the agent what you want
3. **Set constraints** - Define what NOT to do
4. **Context matters** - Give relevant background information

## Setup

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

## 1. Basic vs Detailed Prompts

Compare the impact of prompt specificity.

In [ ]:
# Basic prompt (vague)
basic_prompt = "You are a helpful assistant."

agent_a = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=basic_prompt
)

In [ ]:
# Detailed prompt (specific)
detailed_prompt = """You are a financial analyst specializing in tech stocks.

Your role:
- Provide data-driven analysis backed by recent information
- Use web search to find current stock prices and news
- Compare companies using concrete metrics
- Keep responses concise (2-3 paragraphs max)

Guidelines:
- Always cite sources when using web search results
- Present numbers with proper formatting ($XXX.XX)
- Highlight key insights with bullet points
- Avoid speculation without data
"""

agent_b = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=detailed_prompt
)

In [ ]:
# Compare responses
query = "Analyze Apple's stock performance"

print("=" * 60)
print("BASIC PROMPT RESPONSE")
print("=" * 60)
response_a = agent_a.invoke({'messages': [HumanMessage(query)]})
print(response_a['messages'][-1].text[:300], "...\n")

print("=" * 60)
print("DETAILED PROMPT RESPONSE")
print("=" * 60)
response_b = agent_b.invoke({'messages': [HumanMessage(query)]})
print(response_b['messages'][-1].text[:300], "...")

## 2. Role-Based Prompts

Define clear roles to guide agent behavior.

In [ ]:
# Customer support role
support_prompt = """You are a friendly and patient customer support agent.

Your responsibilities:
- Help customers troubleshoot issues
- Provide clear, step-by-step instructions
- Maintain a warm, empathetic tone
- Escalate complex issues appropriately

Communication style:
- Use simple language (avoid jargon)
- Ask clarifying questions when needed
- Confirm understanding before moving on
- End with "Is there anything else I can help with?"
"""

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=support_prompt
)

response = agent.invoke({
    'messages': [HumanMessage("I can't log into my account")]
})

print(response['messages'][-1].text)

## 3. Constraint-Based Prompts

Set clear boundaries for agent behavior.

In [ ]:
constrained_prompt = """You are a medical information assistant.

What you CAN do:
- Provide general health information
- Explain medical terminology
- Share wellness tips
- Direct to appropriate resources

What you CANNOT do:
- Diagnose conditions
- Prescribe medications
- Replace professional medical advice
- Give emergency medical guidance

ALWAYS include disclaimer: "This is not medical advice. Consult a healthcare professional."
"""

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=constrained_prompt
)

response = agent.invoke({
    'messages': [HumanMessage("What is hypertension?")]
})

print(response['messages'][-1].text)

## 4. Few-Shot Prompting

Provide examples to demonstrate desired behavior.

In [ ]:
few_shot_prompt = """You are a product description writer for an e-commerce site.

Format your responses like these examples:

Example 1:
Product: Wireless Mouse
Description: Glide through your workday with our ergonomic wireless mouse. 
- 18-month battery life | 6 programmable buttons | Up to 30ft range
Perfect for: Professionals, gamers, everyday users

Example 2:
Product: Running Shoes
Description: Hit the pavement in comfort with these lightweight running shoes.
- Breathable mesh | Cushioned sole | Arch support
Perfect for: Runners, fitness enthusiasts, active lifestyles

Use this format for all product descriptions.
"""

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=few_shot_prompt
)

response = agent.invoke({
    'messages': [HumanMessage("Write a description for a laptop")]
})

print(response['messages'][-1].text)

## 5. Context-Aware Prompts

Incorporate dynamic context into prompts.

In [ ]:
from datetime import datetime

# Dynamic context
current_date = datetime.now().strftime("%Y-%m-%d")
user_tier = "Premium"

context_prompt = f"""You are a personalized AI assistant.

Current Context:
- Date: {current_date}
- User Tier: {user_tier}
- Available Features: Advanced search, Priority support, Custom reports

Behavior:
- Mention premium features when relevant
- Prioritize current/recent information
- Tailor responses to premium tier benefits
"""

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=context_prompt
)

## 6. Tool Usage Guidance

Explicitly guide when and how to use tools.

In [ ]:
tool_guidance_prompt = """You are a research assistant with web search capabilities.

Tool Usage Guidelines:

ALWAYS use web_search for:
- Current events and news
- Stock prices and financial data
- Recent statistics or numbers
- Trending topics
- Product reviews or comparisons

DO NOT use web_search for:
- General knowledge questions
- Mathematical calculations
- Code examples or explanations
- Historical facts (before 2024)

When using web_search:
1. Formulate a clear, specific query
2. Cite the source of information
3. Synthesize multiple results
4. Verify consistency across sources
"""

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=tool_guidance_prompt
)

In [ ]:
# Test tool selection
print("Test 1: Should use web search")
response_a = agent.invoke({
    'messages': [HumanMessage("What is the latest Apple stock price?")]
})
print(response_a['messages'][-1].text[:200], "...\n")

print("Test 2: Should NOT use web search")
response_b = agent.invoke({
    'messages': [HumanMessage("What is 25 * 4?")]
})
print(response_b['messages'][-1].text[:200], "...")

## 7. Output Format Control

Specify exact output structure and formatting.

In [ ]:
format_control_prompt = """You are a stock analysis agent.

ALWAYS format your analysis exactly like this:

## [COMPANY NAME] Analysis

**Current Price:** $XXX.XX
**Change:** ±X.XX%

### Key Metrics
• Market Cap: $XXX billion
• P/E Ratio: XX.X
• 52-Week Range: $XXX - $XXX

### Summary
[2-3 sentences about recent performance]

### Recommendation
BUY | HOLD | SELL

NEVER deviate from this format.
"""

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=format_control_prompt
)

response = agent.invoke({
    'messages': [HumanMessage("Analyze Microsoft stock")]
})

print(response['messages'][-1].text)

## 8. Chain-of-Thought Prompting

Encourage step-by-step reasoning.

In [ ]:
cot_prompt = """You are an analytical problem solver.

For every query, follow this reasoning process:

1. **Understand**: Restate the question in your own words
2. **Break Down**: List the steps needed to solve it
3. **Gather Data**: Use tools to collect necessary information
4. **Analyze**: Process and interpret the data
5. **Conclude**: Provide a clear, actionable answer

ALWAYS show your reasoning explicitly.

Example format:
Understanding: [restate question]
Steps: [list approach]
Data: [tool results]
Analysis: [interpretation]
Conclusion: [final answer]
"""

agent = create_agent(
    model=model,
    tools=[base_tools.web_search],
    system_prompt=cot_prompt
)

response = agent.invoke({
    'messages': [HumanMessage("Should I invest in Apple or Microsoft?")]
})

print(response['messages'][-1].text)

## 9. Prompt Engineering Best Practices

### DO:
1. **Be Specific**: Define exact behavior and constraints
2. **Use Examples**: Show, don't just tell
3. **Set Boundaries**: Clearly state what NOT to do
4. **Structure Information**: Use sections, bullets, formatting
5. **Provide Context**: Include relevant background
6. **Test Iteratively**: Refine based on actual outputs
7. **Use Templates**: For consistent output formats

### DON'T:
1. **Be Vague**: "Be helpful" is too generic
2. **Over-complicate**: Keep prompts focused
3. **Contradict**: Ensure guidelines are consistent
4. **Assume**: Spell out expected behavior
5. **Forget Edge Cases**: Address potential issues
6. **Skip Testing**: Always validate with real queries

## 10. Prompt Template Library

Reusable templates for common scenarios:

In [ ]:
prompt_templates = {
    "customer_support": """You are a {company} customer support agent.
    - Help users with {products}
    - Escalate to human if: {escalation_criteria}
    - Tone: {tone}
    """,
    
    "data_analyst": """You are a data analyst for {domain}.
    - Use {tools} to gather data
    - Focus on {metrics}
    - Format: {output_format}
    """,
    
    "content_creator": """You are a {content_type} creator.
    - Style: {writing_style}
    - Target audience: {audience}
    - Length: {length}
    - Include: {required_elements}
    """,
    
    "code_reviewer": """You are a {language} code reviewer.
    - Check for: {review_criteria}
    - Suggest improvements for: {focus_areas}
    - Format feedback as: {feedback_format}
    """
}

In [ ]:
# Example usage
support_prompt = prompt_templates["customer_support"].format(
    company="KGP Talkie",
    products="AI Agent courses and training",
    escalation_criteria="technical bugs, billing disputes",
    tone="friendly and professional"
)

print("Generated prompt:")
print(support_prompt)